# Изучение баз данных книжного сервиса
Проанализировать данные из нескольких баз данных, активность пользователей, количество обзоров, оценок. Все эти оценки требуются для последующего ценностного предложения для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Для красоты выведем схему баз данных

In [2]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url="https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=&cache=v2")

# Задания
* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [3]:
query = ''' select * from books'''
books = pd.io.sql.read_sql(query, con = engine)
display(books.head(3))

query = ''' select * from authors'''
authors = pd.io.sql.read_sql(query, con = engine)
display(authors.head(3))

query = ''' select * from publishers'''
publishers = pd.io.sql.read_sql(query, con = engine)
display(publishers.head(3))

query = ''' select * from reviews'''
reviews = pd.io.sql.read_sql(query, con = engine)
display(reviews.head(3))

query = ''' select * from ratings'''
ratings = pd.io.sql.read_sql(query, con = engine)
display(ratings.head(3))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [4]:
for i in [books, authors, publishers, reviews, ratings]:
    print('='*30)
    i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        

Ознакомились со всеми таблицами, все отображается корректно, данные полные, никаких проблем не обнаружено, приступаем к заданиям

# Задание 1 
<b>Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
query = ''' 
SELECT COUNT(book_id)
FROM books
WHERE publication_date>'2000-01-01'
'''
task1 = pd.io.sql.read_sql(query, con = engine)
print('С 1 января 2000 года вышло {} книг.'.format(task1['count'][0]))

С 1 января 2000 года вышло 819 книг.


# Задание 2 
<b>Для каждой книги посчитайте количество обзоров и среднюю оценку

In [6]:
query = ''' 
SELECT b.book_id,b.title,COUNT(DISTINCT(rev.review_id)),AVG(rat.rating)
FROM books AS b
LEFT JOIN reviews AS rev ON rev.book_id=b.book_id
LEFT JOIN ratings AS rat ON rat.book_id=b.book_id
GROUP BY b.book_id
'''
task2 = pd.io.sql.read_sql(query, con = engine)
task2.sort_values('avg',ascending=False)

,book_id,title,count,avg
901,902,The War of Art: Break Through the Blocks & Win...,2,5.00
937,938,Triptych (Will Trent #1),2,5.00
517,518,Pop Goes the Weasel (Alex Cross #5),2,5.00
168,169,Crucial Conversations: Tools for Talking When ...,2,5.00
731,732,The Ghost Map: The Story of London's Most Terr...,2,5.00
...,...,...,...,...
914,915,The World Is Flat: A Brief History of the Twen...,3,2.25
315,316,His Excellency: George Washington,2,2.00
370,371,Junky,2,2.00
201,202,Drowning Ruth,3,2.00


<b>Вывод</b>

Теперь у нас есть статистика по книгам с обзорами и средней оценкой, с помощью этих данных можно будет найти популярные книги.
При двух обзорах, оценка может быть как самой высокой, так и самой низкой.

# Задание 3 
<b>Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [7]:
query = ''' 
SELECT b.publisher_id, p.publisher, COUNT(b.book_id)
FROM books AS b
LEFT JOIN publishers AS p ON p.publisher_id=b.publisher_id
WHERE b.num_pages>50
GROUP BY b.publisher_id, p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1
'''
task3 = pd.io.sql.read_sql(query, con = engine)
print('Издательство {} выпустило наибольшее количество книг {}, без учета брошюр.'.format(task3['publisher'][0],task3['count'][0]))

Издательство Penguin Books выпустило наибольшее количество книг 42, без учета брошюр.


# Задание 4 
<b>Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

<b>Вариант 1</b> Просто средняя оценка книг 

In [8]:
query = ''' 
SELECT a.author,COUNT(r.rating_id) AS count_ratings, AVG(r.rating) AS average_rating
FROM books AS b
LEFT JOIN authors AS a ON b.author_id=a.author_id
LEFT JOIN ratings AS r ON r.book_id=b.book_id
GROUP BY a.author, b.book_id
HAVING COUNT(r.rating_id)>=50
ORDER BY AVG(r.rating) DESC
LIMIT 3
'''
task4 = pd.io.sql.read_sql(query, con = engine)
display(task4)
print('Автор {} с самой высокой средней оценкой книг {}, оценок поставили {}'.format(task4['author'][0],task4['average_rating'][0],task4['count_ratings'][0]))

,author,count_ratings,average_rating
0,J.K. Rowling/Mary GrandPré,82,4.414634
1,J.R.R. Tolkien,74,4.391892
2,J.K. Rowling/Mary GrandPré,80,4.287500


Автор J.K. Rowling/Mary GrandPré с самой высокой средней оценкой книг 4.414634146341464, оценок поставили 82


<b>Вариант 2</b> Средняя от средних оценок книг

In [9]:
query = ''' SELECT avg.author, AVG(avg.average_rating) AS average
FROM (
SELECT a.author,COUNT(r.rating_id) AS count_ratings, AVG(r.rating) AS average_rating
FROM books AS b
LEFT JOIN authors AS a ON b.author_id=a.author_id
LEFT JOIN ratings AS r ON r.book_id=b.book_id
GROUP BY a.author, b.book_id
HAVING COUNT(r.rating_id)>=50
ORDER BY AVG(r.rating) DESC
) AS avg
GROUP BY avg.author
ORDER BY average DESC
LIMIT 3
'''
task4 = pd.io.sql.read_sql(query, con = engine)
display(task4)
print('Автор {} с самой высокой средней оценкой книг {}'.format(task4['author'][0],task4['average'][0]))

,author,average
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446


Автор J.K. Rowling/Mary GrandPré с самой высокой средней оценкой книг 4.283844038868471


<b>Вывод</b>

Как в одном, так и во втором варианте получает самую высокую оценку J.K. Rowling/Mary GrandPré

# Задание 5 
<b>Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [10]:
query = ''' 
SELECT AVG(rev.average_reviews) AS average_reviews FROM(
    SELECT  COUNT(review_id) AS average_reviews
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id)>50
        )
    GROUP BY username) AS rev
'''
task5 = pd.io.sql.read_sql(query, con = engine)
print('Среднее количество обзоров от пользователей с более 50 оценками равно {}'.format(round(task5['average_reviews'][0],2)))

Среднее количество обзоров от пользователей с более 50 оценками равно 24.33


# Общий вывод
* Имеется информация о 1000 книгах
* 819 книг из 1000 были изданы после 1 января 2000
* Максимальная и минимальная оценка книг 5 и 1.5 соответственно
* Без учета брошюр Penguin Books выпустило наибольшее количество книг 42
* Автором с самыми высокими оценками является J.K. Rowling/Mary GrandPré
* Среднее количество обзоров от пользователей с более 50 оценками равно 24